In [1]:
import subprocess
import os
import time

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

login('YourTokenHere')

/root/miniconda3/envs/gemma/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [5]:
# Define the checkpoint directory
checkpoint_path = "/root/autodl-tmp/Projects/Model_Checkpoints/Checkpoints_Xuezha333_Z/gemma-qlora-re_2024-04-18_10-51-12/checkpoint-6000"  # Replace with your checkpoint path

# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path,
                                             quantization_config=bnb_config,
                                             device_map="cuda:0"
                                             )


Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.70s/it]


# yz's eval starts here

In [6]:
# read test file
# or all test data in a list
import json
with open('/root/autodl-tmp/Projects/Gemma_RE/gemma_re_z/semeval/semeval_test_processed_cloze.json', 'r') as file:
    test_data = [json.loads(line) for line in file.readlines()]

In [7]:
print(test_data[0], type(test_data[0]))

{'prompt': 'the most common <h> audits </h> about <t> waste </t> and recycling .', 'completion': {'h': 'audits', 't': 'waste', 'relation': 'Message-Topic(e1,e2)'}} <class 'dict'>


In [8]:
# predict
from tqdm import trange
test_predict = []
time_predict = []
for i in trange(len(test_data)):
    time.sleep(0.1)
    question = test_data[i]['prompt']
    formatted_prompt = f"### Question: {question}\n### Answer:"
    input_ids = tokenizer.encode(formatted_prompt, return_tensors='pt').to(device)
    outputs = model.generate(input_ids, max_length=120, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
    s = time.time()
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    t = time.time()
    time_predict.append(t-s)
    test_predict.append(generated_text)

100%|██████████| 2717/2717 [1:44:46<00:00,  2.31s/it]


In [9]:
print(test_predict[0], type(test_predict[0]))

### Question: the most common <h> audits </h> about <t> waste </t> and recycling .
### Answer: {'h': 'audits', 't': 'waste', 'relation': 'Message-Topic(e1,e2)'} <class 'str'>


In [11]:
# save predict file
with open('data/test_predict_data_c6k.json', 'w', encoding='utf-8') as file:
    json.dump(test_predict, file, ensure_ascii=False)

In [12]:
test_predict_ans = []
for line in test_predict:
    n = line.find('### Answer: ')
    line = line[n+11:]
    head_start = line.find("'head': ")
    head_line = line[head_start+9:]
    head_end = head_line.find("'")
    head = head_line[:head_end]
    tail_start = line.find("'tail': ")
    tail_line = line[tail_start+9:]
    tail_end = tail_line.find("'")
    tail = tail_line[:tail_end]
    relation_start = line.find("'relation': ")
    relation_line = line[relation_start+13:]
    relation_end = relation_line.find("'")
    relation = relation_line[:relation_end]
    test_predict_ans.append({'head':head,'tail':tail,'relation':relation})
print(test_predict_ans[0], type(test_predict_ans[0]))

{'head': 'audits', 'tail': 'audits', 'relation': 'Message-Topic(e1,e2)'} <class 'dict'>


In [13]:
'''test_data = ['{"token": ["the", "most", "common", "audits", "were", "about", "waste", "and", "recycling", "."], "h": {"name": "audits", "pos": [3, 4]}, "t": {"name": "waste", "pos": [6, 7]}, "relation": "Message-Topic(e1,e2)"}',
'{"token": ["this", "thesis", "defines", "the", "clinical", "characteristics", "of", "amyloid", "disease", "."], "h": {"name": "thesis", "pos": [1, 2]}, "t": {"name": "clinical characteristics", "pos": [4, 6]}, "relation": "Message-Topic(e1,e2)"}',
'{"token": ["this", "outline", "focuses", "on", "spirituality", ",", "esotericism", ",", "mysticism", ",", "religion", "and/or", "parapsychology", "."], "h": {"name": "outline", "pos": [1, 2]}, "t": {"name": "spirituality", "pos": [4, 5]}, "relation": "Message-Topic(e1,e2)"}',
'{"token": ["many", "of", "his", "literary", "pieces", "narrate", "and", "mention", "stories", "that", "took", "place", "in", "lipa", "."], "h": {"name": "pieces", "pos": [4, 5]}, "t": {"name": "stories", "pos": [8, 9]}, "relation": "Other"}',
'{"token": ["news", "programs", "commented", "on", "the", "violence", "from", "the", "game", "and", "expressed", "worries", "on", "how", "it", "would", "affect", "the", "players", "\'", "personalities", "."], "h": {"name": "news programs", "pos": [0, 2]}, "t": {"name": "violence", "pos": [5, 6]}, "relation": "Message-Topic(e1,e2)"}',
'{"token": ["in", "the", "article", ",", "the", "authors", "explore", "the", "use", "of", "technology", "in", "small", "pharmacy", "chains", "."], "h": {"name": "article", "pos": [2, 3]}, "t": {"name": "use", "pos": [8, 9]}, "relation": "Message-Topic(e1,e2)"}',
'{"token": ["the", "magazine", "was", "founded", "in", "order", "to", "keep", "athletes", "serving", "as", "soldiers", "informed", "about", "their", "sport", "back", "home", "."], "h": {"name": "magazine", "pos": [1, 2]}, "t": {"name": "sport", "pos": [15, 16]}, "relation": "Other"}']

test_predict = ['{"token": ["the", "most", "common", "audits", "were", "about", "waste", "and", "recycling", "."], "h": {"name": "audits", "pos": [3, 4]}, "t": {"name": "waste", "pos": [6, 7]}, "relation": "Message-Topic(e1,e2)"}',
'{"token": ["this", "thesis", "defines", "the", "clinical", "characteristics", "of", "amyloid", "disease", "."], "h": {"name": "the", "pos": [1, 2]}, "t": {"name": "clinical characteristics", "pos": [4, 6]}, "relation": "Message-Topic(e1,e2)"}',
'{"token": ["this", "outline", "focuses", "on", "spirituality", ",", "esotericism", ",", "mysticism", ",", "religion", "and/or", "parapsychology", "."], "h": {"name": "outline", "pos": [1, 2]}, "t": {"name": "spirituality", "pos": [4, 5]}, "relation": "Message-Topic(e1,e2)"}',
'{"token": ["many", "of", "his", "literary", "pieces", "narrate", "and", "mention", "stories", "that", "took", "place", "in", "lipa", "."], "h": {"name": "pieces", "pos": [4, 5]}, "t": {"name": "stories", "pos": [8, 9]}, "relation": "Message-Topic(e1,e2)"}',
'{"token": ["news", "programs", "commented", "on", "the", "violence", "from", "the", "game", "and", "expressed", "worries", "on", "how", "it", "would", "affect", "the", "players", "\'", "personalities", "."], "h": {"name": "news programs", "pos": [0, 2]}, "t": {"name": "violence", "pos": [5, 6]}, "relation": "Other"}',
'{"token": ["in", "the", "article", ",", "the", "authors", "explore", "the", "use", "of", "technology", "in", "small", "pharmacy", "chains", "."], "h": {"name": "article", "pos": [2, 3]}, "t": {"name": "use", "pos": [8, 9]}, "relation": "Other"}',
'{"token": ["the", "magazine", "was", "founded", "in", "order", "to", "keep", "athletes", "serving", "as", "soldiers", "informed", "about", "their", "sport", "back", "home", "."], "h": {"name": "magazine", "pos": [1, 2]}, "t": {"name": "sport", "pos": [15, 16]}, "relation": "Other"}']'''


'test_data = [\'{"token": ["the", "most", "common", "audits", "were", "about", "waste", "and", "recycling", "."], "h": {"name": "audits", "pos": [3, 4]}, "t": {"name": "waste", "pos": [6, 7]}, "relation": "Message-Topic(e1,e2)"}\',\n\'{"token": ["this", "thesis", "defines", "the", "clinical", "characteristics", "of", "amyloid", "disease", "."], "h": {"name": "thesis", "pos": [1, 2]}, "t": {"name": "clinical characteristics", "pos": [4, 6]}, "relation": "Message-Topic(e1,e2)"}\',\n\'{"token": ["this", "outline", "focuses", "on", "spirituality", ",", "esotericism", ",", "mysticism", ",", "religion", "and/or", "parapsychology", "."], "h": {"name": "outline", "pos": [1, 2]}, "t": {"name": "spirituality", "pos": [4, 5]}, "relation": "Message-Topic(e1,e2)"}\',\n\'{"token": ["many", "of", "his", "literary", "pieces", "narrate", "and", "mention", "stories", "that", "took", "place", "in", "lipa", "."], "h": {"name": "pieces", "pos": [4, 5]}, "t": {"name": "stories", "pos": [8, 9]}, "relation": 

In [13]:
import pandas as pd
data = pd.DataFrame({'ans':test_data,'predict':test_predict_ans})
data

,ans,predict
0,{'prompt': 'the most common <h> audits </h> ab...,"{'head': 'audits', 'tail': 'audits', 'relation..."
1,{'prompt': 'this <h> thesis </h> the <t> clini...,"{'head': 'thesis', 'tail': 'thesis', 'relation..."
2,{'prompt': 'this <h> outline </h> on <t> spiri...,"{'head': 'outline', 'tail': 'outline', 'relati..."
3,{'prompt': 'many of his literary <h> pieces </...,"{'head': 'pieces', 'tail': 'pieces', 'relation..."
4,{'prompt': '<h> news programs </h> on the <t> ...,"{'head': 'news programs', 'tail': 'news progra..."
...,...,...
2712,{'prompt': 'congress <h> workers </h> with col...,"{'head': 'workers', 'tail': 'workers', 'relati..."
2713,{'prompt': 'the teacher always assigned <h> st...,"{'head': 'students', 'tail': 'students', 'rela..."
2714,{'prompt': 'the <h> dean </h> the <t> faculty ...,"{'head': 'dean', 'tail': 'dean', 'relation': '..."
2715,{'prompt': 'and they learned that lenin was th...,"{'head': 'leader', 'tail': 'leader', 'relation..."


In [15]:
#len(groups[groups['ans']=='1'])

In [16]:
#groups[(groups['ans']=='1') & (groups['predict']=='1')]

In [14]:
a_relation = []
p_relation = []
for i in range(len(data)):
    a_relation.append(data['ans'][i]['completion']['relation'])
    p_relation.append(data['predict'][i]['relation'])
data['a_relation'] = a_relation
data['p_relation'] = p_relation
data


,ans,predict,a_relation,p_relation
0,{'prompt': 'the most common <h> audits </h> ab...,"{'head': 'audits', 'tail': 'audits', 'relation...","Message-Topic(e1,e2)","Message-Topic(e1,e2)"
1,{'prompt': 'this <h> thesis </h> the <t> clini...,"{'head': 'thesis', 'tail': 'thesis', 'relation...","Message-Topic(e1,e2)","Message-Topic(e1,e2)"
2,{'prompt': 'this <h> outline </h> on <t> spiri...,"{'head': 'outline', 'tail': 'outline', 'relati...","Message-Topic(e1,e2)","Message-Topic(e1,e2)"
3,{'prompt': 'many of his literary <h> pieces </...,"{'head': 'pieces', 'tail': 'pieces', 'relation...","Message-Topic(e1,e2)","Message-Topic(e1,e2)"
4,{'prompt': '<h> news programs </h> on the <t> ...,"{'head': 'news programs', 'tail': 'news progra...","Message-Topic(e1,e2)","Message-Topic(e1,e2)"
...,...,...,...,...
2712,{'prompt': 'congress <h> workers </h> with col...,"{'head': 'workers', 'tail': 'workers', 'relati...","Member-Collection(e1,e2)","Entity-Origin(e1,e2)"
2713,{'prompt': 'the teacher always assigned <h> st...,"{'head': 'students', 'tail': 'students', 'rela...","Member-Collection(e1,e2)","Member-Collection(e1,e2)"
2714,{'prompt': 'the <h> dean </h> the <t> faculty ...,"{'head': 'dean', 'tail': 'dean', 'relation': '...","Member-Collection(e1,e2)","Member-Collection(e1,e2)"
2715,{'prompt': 'and they learned that lenin was th...,"{'head': 'leader', 'tail': 'leader', 'relation...","Member-Collection(e1,e2)","Member-Collection(e1,e2)"


In [31]:
data.iloc[919]['predict']

{'head': '### Question: the <h> man </h> on the <t> roof </t> all day .\n### Answer: {',
 'tail': '### Question: the <h> man </h> on the <t> roof </t> all day .\n### Answer: {',
 'relation': 'Question: the <h> man </h> on the <t> roof </t> all day .\n### Answer: {'}

In [32]:
x=[]
for i in range(len(data)):
    if data['a_relation'][i] not in rlt:
        print("Error:a_relation"+str(i))
    if data['p_relation'][i] not in rlt:
        print("Error:p_relation row"+str(i))
        print(data['p_relation'][i])
        x.append(i)
print(x)

Error:p_relation row454
Material-Source(e1,e2)
Error:p_relation row850
Source-Destination(e1,e2)
Error:p_relation row919
Question: the <h> man </h> on the <t> roof </t> all day .
### Answer: {
Error:p_relation row960
Element-Whole(e1,e2)
Error:p_relation row1409
ederacy
Error:p_relation row1437
Entry-into-force(e1,e2)
Error:p_relation row1487
Ingredient-Producer(e1,e2)
Error:p_relation row1534
Time-Oriented
Error:p_relation row1546
Material-Origin(e1,e2)
Error:p_relation row1617
Timescale-Other
Error:p_relation row1625
Condition-Status
Error:p_relation row1686
Time-Series(e1,e2)
Error:p_relation row1796
Ingredient-Origin(e1,e2)
Error:p_relation row1836
Time-Duration(e1,e2)
Error:p_relation row1864
Emotion-Event(e1,e2)
Error:p_relation row1876
Element-of-Collection(e1,e2)
Error:p_relation row1894
Starting-At(e1,e2)
Error:p_relation row1965
From-Entity-Origin(e1,e2)
Error:p_relation row2015
Origin-Source(e1,e2)
Error:p_relation row2026
Time-Stamp ( e1 , e2)
Error:p_relation row2051
Messa

In [36]:
data2 = data.iloc[x]
data2.to_csv('data/weird_predict.csv')

In [15]:
# all kind of relation extraction
rlt = ["Component-Whole(e2,e1)",
    "Other",
    "Instrument-Agency(e2,e1)",
    "Member-Collection(e1,e2)",
    "Cause-Effect(e2,e1)",
    "Entity-Destination(e1,e2)",
    "Content-Container(e1,e2)",
    "Message-Topic(e1,e2)",
    "Product-Producer(e2,e1)",
    "Member-Collection(e2,e1)",
    "Entity-Origin(e1,e2)",
    "Cause-Effect(e1,e2)",
    "Component-Whole(e1,e2)",
    "Message-Topic(e2,e1)",
    "Product-Producer(e1,e2)",
    "Entity-Origin(e2,e1)",
    "Content-Container(e2,e1)",
    "Instrument-Agency(e1,e2)",
    "Entity-Destination(e2,e1)"]
#rlt = ['Other','Message-Topic(e1,e2)']

##### 在xxx关系的eval中：
correct_by_relation表示：关系相同，进行计数（TP）；

guessed_by_relation表示：预测显示有关系则全部进行计数（TP+FP）；

gold_by_relation：实际标签显示有关系则全部进行计数（TP+FN）。

In [19]:
'''ans = data['ans'][0]
ans['h']'''

"ans = data['ans'][0]\nans['h']"

In [16]:
# TP, TP+FP, TP+TN
result = dict()
#r = "Message-Topic(e1,e2)"
for r in rlt:
    guessed_by_relation = len(data[data['p_relation']==r])
    gold_by_relation = len(data[data['a_relation']==r])
    #same_relation_diff_element = 0
    df = data[(data['p_relation']==r) & (data['a_relation']==r)].reset_index(drop=True)
    correct_by_relation = len(df)
    result[r] = [correct_by_relation, guessed_by_relation, gold_by_relation] # TP, TP+FP, TP+FN

print(result)


{'Component-Whole(e2,e1)': [125, 181, 150], 'Other': [151, 257, 454], 'Instrument-Agency(e2,e1)': [115, 160, 134], 'Member-Collection(e1,e2)': [25, 46, 32], 'Cause-Effect(e2,e1)': [166, 194, 194], 'Entity-Destination(e1,e2)': [245, 330, 291], 'Content-Container(e1,e2)': [91, 101, 153], 'Message-Topic(e1,e2)': [178, 206, 210], 'Product-Producer(e2,e1)': [107, 139, 123], 'Member-Collection(e2,e1)': [189, 239, 201], 'Entity-Origin(e1,e2)': [154, 270, 211], 'Cause-Effect(e1,e2)': [116, 135, 134], 'Component-Whole(e1,e2)': [140, 177, 162], 'Message-Topic(e2,e1)': [27, 32, 51], 'Product-Producer(e1,e2)': [82, 105, 108], 'Entity-Origin(e2,e1)': [31, 33, 47], 'Content-Container(e2,e1)': [34, 42, 39], 'Instrument-Agency(e1,e2)': [19, 46, 22], 'Entity-Destination(e2,e1)': [0, 1, 1]}


In [17]:
# evaluation
evaluation = dict()
for k in result:
    precision = round(result[k][0]/result[k][1],2)
    recall = round(result[k][0]/result[k][2],2)
    if precision+recall == 0:
        f1 = 0
    else:
        f1 = round((2*precision*recall)/(precision+recall),2)
    evaluation[k] = [precision, recall, f1] #p, r, f1

print(evaluation)

{'Component-Whole(e2,e1)': [0.69, 0.83, 0.75], 'Other': [0.59, 0.33, 0.42], 'Instrument-Agency(e2,e1)': [0.72, 0.86, 0.78], 'Member-Collection(e1,e2)': [0.54, 0.78, 0.64], 'Cause-Effect(e2,e1)': [0.86, 0.86, 0.86], 'Entity-Destination(e1,e2)': [0.74, 0.84, 0.79], 'Content-Container(e1,e2)': [0.9, 0.59, 0.71], 'Message-Topic(e1,e2)': [0.86, 0.85, 0.85], 'Product-Producer(e2,e1)': [0.77, 0.87, 0.82], 'Member-Collection(e2,e1)': [0.79, 0.94, 0.86], 'Entity-Origin(e1,e2)': [0.57, 0.73, 0.64], 'Cause-Effect(e1,e2)': [0.86, 0.87, 0.86], 'Component-Whole(e1,e2)': [0.79, 0.86, 0.82], 'Message-Topic(e2,e1)': [0.84, 0.53, 0.65], 'Product-Producer(e1,e2)': [0.78, 0.76, 0.77], 'Entity-Origin(e2,e1)': [0.94, 0.66, 0.78], 'Content-Container(e2,e1)': [0.81, 0.87, 0.84], 'Instrument-Agency(e1,e2)': [0.41, 0.86, 0.56], 'Entity-Destination(e2,e1)': [0.0, 0.0, 0]}


In [18]:
# macro p, r, f1
p_macro_lst = [value[0] for value in evaluation.values()]
p_macro = round(sum(p_macro_lst)/len(p_macro_lst),2)
r_macro_lst = [value[1] for value in evaluation.values()]
r_macro = round(sum(r_macro_lst)/len(r_macro_lst),2)
f1_macro_lst = [value[2] for value in evaluation.values()]
f1_macro = round(sum(f1_macro_lst)/len(f1_macro_lst),2)
print('p_macro: ' + str(p_macro))
print('r_macro: ' + str(r_macro))
print('f1_macro: ' + str(f1_macro))

p_macro: 0.71
r_macro: 0.73
f1_macro: 0.71


In [19]:
# micro p, r, f1
TP_lst = [value[0] for value in result.values()]
TPFP_lst = [value[1] for value in result.values()]
TPTN_lst = [value[2] for value in result.values()]
p_micro = round(sum(TP_lst)/sum(TPFP_lst),2)
r_micro = round(sum(TP_lst)/sum(TPTN_lst),2)
f1_micro = round((2*p_micro*r_micro)/(p_micro+r_micro),2)
print('p_micro: ' + str(p_micro))
print('r_micro: ' + str(r_micro))
print('f1_micro: ' + str(f1_micro))

p_micro: 0.74
r_micro: 0.73
f1_micro: 0.73


In [21]:
# result for each relation
pd_result = pd.DataFrame(result.items(), columns=['relation','result'])
pd_result[['TP','TP+FP','TP+TN']] = pd.DataFrame(pd_result['result'].tolist())
pd_result = pd_result.drop('result', axis=1)
pd_result

,relation,TP,TP+FP,TP+TN
0,"Component-Whole(e2,e1)",125,181,150
1,Other,151,257,454
2,"Instrument-Agency(e2,e1)",115,160,134
3,"Member-Collection(e1,e2)",25,46,32
4,"Cause-Effect(e2,e1)",166,194,194
5,"Entity-Destination(e1,e2)",245,330,291
6,"Content-Container(e1,e2)",91,101,153
7,"Message-Topic(e1,e2)",178,206,210
8,"Product-Producer(e2,e1)",107,139,123
9,"Member-Collection(e2,e1)",189,239,201


In [25]:
pd_result.to_csv('data/test_predict_result_c6k.csv',index=False)

ModuleNotFoundError: No module named 'openpyxl'

In [23]:
# evaluation report for each relation
pd_eval = pd.DataFrame(evaluation.items(), columns=['relation','eval'])
pd_eval[['precision','recall','f1']] = pd.DataFrame(pd_eval['eval'].tolist())
pd_eval = pd_eval.drop('eval', axis=1)
pd_eval

,relation,precision,recall,f1
0,"Component-Whole(e2,e1)",0.69,0.83,0.75
1,Other,0.59,0.33,0.42
2,"Instrument-Agency(e2,e1)",0.72,0.86,0.78
3,"Member-Collection(e1,e2)",0.54,0.78,0.64
4,"Cause-Effect(e2,e1)",0.86,0.86,0.86
5,"Entity-Destination(e1,e2)",0.74,0.84,0.79
6,"Content-Container(e1,e2)",0.90,0.59,0.71
7,"Message-Topic(e1,e2)",0.86,0.85,0.85
8,"Product-Producer(e2,e1)",0.77,0.87,0.82
9,"Member-Collection(e2,e1)",0.79,0.94,0.86


In [24]:
pd_eval.to_csv('data/test_predict_eval_c6k.csv',index=False)